# Setting up the computational model
To simulate the motion of molecules according to the kinetic theory of gases, we need to set up a dynamic n-body simulation with collisions. Note that the objective of this article is to produce a simulation for pedagogical purposes. Therefore, the code is set up to maximize understanding, not execution speed.

## Defining molecular properties
We start by importing all the required modules.

In [1]:
import os
import sys
import time
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation